In [1]:
import requests
import pandas as pd
import glob
import os

In [37]:
# PROVINCIAL SOLO LIMA METROPOLITANA
from numpy import NAN


url = "https://api.resultadoserm2022.onpe.gob.pe/results/03/140100"
headers = {
    "User-Agent": "Mozilla/5.0 (Linux; Android 6.0; Nexus 5 Build/MRA58N) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/106.0.0.0 Mobile Safari/537.36",
}

resp = requests.get(url, headers=headers)

data = resp.json()['results']

fecha = resp.json()['generals']['actData']['FECHA']
hora = resp.json()['generals']['actData']['HORA']
avance = resp.json()['generals']['actData']['POR_AVANCE']

if data != []:
    df = pd.DataFrame(data)
    df['fecha'] = fecha
    df['hora'] = hora
    df['avance'] = avance
    df = df[~(df['C_CODI_AGP'].isin(['80','81']))]
    df = df.dropna(subset=['C_CODI_AGP'])
    df.to_csv(f"lima_metropolitana.csv")

In [39]:
# REGIONAL
for departamento in range(1,26):
    url = f"https://api.resultadoserm2022.onpe.gob.pe/results/01/{departamento:02d}0000"
    headers = {
        "User-Agent": "Mozilla/5.0 (Linux; Android 6.0; Nexus 5 Build/MRA58N) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/106.0.0.0 Mobile Safari/537.36",
    }

    resp = requests.get(url, headers=headers)

    data = resp.json()['results']

    fecha = resp.json()['generals']['actData']['FECHA']
    hora = resp.json()['generals']['actData']['HORA']
    avance = resp.json()['generals']['actData']['POR_AVANCE']

    if data != []:
        df = pd.DataFrame(data)
        df['departamento'] = f'{departamento:02d}'
        df['fecha'] = fecha
        df['hora'] = hora
        df['avance'] = avance
        df = df[~(df['C_CODI_AGP'].isin(['80','81']))]
        df = df.dropna(subset=['C_CODI_AGP'])
        df.to_csv(f"resultados_regionales/{departamento:02d}0000.csv")


Resultant CSV after joining all CSV files at a particular location...


,C_CODI_AGP,AGRUPACION,IMAGENAGRUPACION,PORC_AVANCE_ACTA,ACTAS_COMPUTADA,TOTAL_MESAS,TOTAL_VOTOS,NLISTA,POR_VALIDOS,POR_EMITIDOS,departamento,fecha,hora,avance
0,99,PIURA RENACE,NaN,100.0,"3,151","3,151","32,151",00000099.jpg,5.214,4.341,19,03/10/2022,08:46 h,64.9022
1,98,ORGANIZACION POLITICA UNIDAD REGIONAL,NaN,100.0,"3,151","3,151","158,102",00000098.jpg,25.638,21.346,19,03/10/2022,08:46 h,64.9022
2,95,MOVIMIENTO INDEPENDIENTE FUERZA REGIONAL,NaN,100.0,"3,151","3,151","75,977",00000095.jpg,12.321,10.258,19,03/10/2022,08:46 h,64.9022
3,97,REGION PARA TODOS,NaN,100.0,"3,151","3,151","102,514",00000097.jpg,16.624,13.841,19,03/10/2022,08:46 h,64.9022
4,96,CONTIGO REGION,NaN,100.0,"3,151","3,151","98,120",00000096.jpg,15.911,13.247,19,03/10/2022,08:46 h,64.9022
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
198,53,MOVIMIENTO POLITICO CAMBIEMOS X HCO,NaN,100.0,"1,807","1,807","52,115",00000053.jpg,16.553,13.554,9,03/10/2022,08:46 h,84.6767
199,54,MOVIMIENTO INDEPENDIENTE REGIONAL MI BUEN VECINO,NaN,100.0,"1,807","1,807","102,896",00000054.jpg,32.683,26.761,9,03/10/2022,08:46 h,84.6767
200,57,MOVIMIENTO INDEPENDIENTE REGIONAL HUANUCO PRIMERO,NaN,100.0,"1,807","1,807","71,692",00000057.jpg,22.772,18.645,9,03/10/2022,08:46 h,84.6767
201,11,PARTIDO FRENTE DE LA ESPERANZA 2021,NaN,100.0,"1,807","1,807","3,921",00000011.jpg,1.245,1.020,9,03/10/2022,08:46 h,84.6767


In [7]:
# setting the path for joining multiple files
files = os.path.join("resultados_regionales", "*.csv")

# list of merged files returned
files = glob.glob(files)

print("Resultant CSV after joining all CSV files at a particular location...");

li = []

for filename in files:
    df = pd.read_csv(filename, index_col=None, header=0, dtype=str)
    li.append(df)

frame = pd.concat(li, axis=0, ignore_index=True).drop(columns=['Unnamed: 0'])
frame.to_csv('resultados_regional.csv')

# # joining files with concat and read_csv
# df_resultados = pd.concat(map(pd.read_csv, files), ignore_index=True).drop(columns=['Unnamed: 0'])
# df_resultados.to_csv('resultados_regional.csv')


Resultant CSV after joining all CSV files at a particular location...
